In [1]:
import import_ipynb
import numpy as np
import sys
import csv
import numpy.ma as ma
import matplotlib.pyplot as plt
import a2_extract_list_meetings as a2
import c1_extract_paragraphe_issue as c1
import c2_extract_sentence_issue as c2
import e1_extract_occ_gen as e1
import e2_extract_occ_original as e2 
import g1_generate_dictionary as g1

importing Jupyter notebook from a2_extract_list_meetings.ipynb
importing Jupyter notebook from a1_compute_list_meeting.ipynb
importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from e1_extract_occ_gen.ipynb
importing Jupyter notebook from e2_extract_occ_original.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb


In [2]:
def open_file(file):
    """Open file that contains all meetings with their information extrated. """
    list_file = []
    f = open(file)
    for x in csv.reader(f):
        list_file.append(x)
    return list_file[1:]

In [4]:
def combine_gen(dictionary, original_occ, generated_occ):
    """For each entities in original dataset, find map entities generated related to it and take interventions of each."""
    combined = []
    for eo in original_occ:
        list_eo = []
        for eg in generated_occ:
            if(eg[1] in dictionary[eo[0]]):
                list_eo.append(eg)
        combined.append(list_eo)
    return combined

In [5]:
def create_stat_egals(generated_occ, original_occ,number):
    """Keep only interventions of one entity in generated that correspond to one in the original dataset. Take the one with the biggest number of interventions. """
    original = []
    generated = []
    for i in range(len(original_occ)):
        original.append((number,original_occ[i][0],original_occ[i][1]))
        g = sorted(generated_occ[i],key=lambda x: x[2], reverse=True)[0]

        generated.append(g)
    return original,generated 

In [6]:
def extract_interventions(number, list_sentences):
    """Extract the intervention for both dataset and keep only the one that match the dictionary"""
    #Load the occurences for the original and generated file, and the dictionary
    original_occ = e2.extract_occurences_issue(number)

    original_occ = [x for x in original_occ if x[0] != 'Korea, Democratic Rep.']

    gen_occ = e1.extract_occurences_issue_ENB(list_sentences, number)

    dictionary = g1.compute_dictionary()

    #Combine element same key
    generated_occ= combine_gen(dictionary, original_occ, gen_occ)

    # Original and generated occurence with only the entities into the dictionary
    original, generated = create_stat_egals(generated_occ, original_occ, number)

    return original, generated

In [7]:
def extract_all_intervention():
    """Extract the intervention for all the issues in the original dataset and write them into .csv file"""
    # Extract all the issue number in the original dataset
    list_meetings = open_file('Text/list_meetings.csv')
    list_issues = a2.issues_list(list_meetings)

    list_generated = []
    list_original = []

    for i in list_issues:
        if(45< int(i) and int(i) < 594):
            list_paragraphes = c1.extract_paragraphes_from_issue(int(i))
            list_sentences = c2.sentences(list_paragraphes)
            original,generated = extract_interventions(int(i), list_sentences)

            list_generated += generated
            
            list_original += original
    return list_generated, list_original

    